In [1]:
import sys
sys.path.append('..')
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import Counter
from collections import defaultdict

# Load the Data

In [2]:
# Load the data from the Excel file
df = pd.read_excel('players.xlsx')

# Extract player data
players = []
for _, row in df.iterrows():
    players.append((row['Name'], row['Position'], row['Skill'], row['Salary (â‚¬M)']))

In [3]:
df = pd.DataFrame(players, columns=['Player Name', 'Position', 'Rating', 'Performance'])
print(df)

         Player Name Position  Rating  Performance
0        Alex Carter       GK      85           90
1       Jordan Smith       GK      88          100
2      Ryan Mitchell       GK      83           85
3     Chris Thompson       GK      80           80
4    Blake Henderson       GK      87           95
5      Daniel Foster      DEF      90          110
6      Lucas Bennett      DEF      85           90
7        Owen Parker      DEF      88          100
8       Ethan Howard      DEF      80           70
9         Mason Reed      DEF      82           75
10      Logan Brooks      DEF      86           95
11      Caleb Fisher      DEF      84           85
12     Nathan Wright      MID      92          120
13      Connor Hayes      MID      89          105
14      Dylan Morgan      MID      91          115
15     Hunter Cooper      MID      83           85
16     Austin Torres      MID      82           80
17  Gavin Richardson      MID      87           95
18      Spencer Ward      MID  

# Problem Definition

``Sports League Optimization``

**Objective** - Assign  players  to  teams according to specified positions in  a  way  that  ensures a balanced distribution of talent while staying within salary caps. (5 teams, 7 players each)

**Players** - Defined by a set of attributes that include:
- Skill rating: Represents the player's ability/talent. 
- Cost: The player's salary (in milions). 
- Position: Goalkeeper  (GK),  Defender  (DEF),  Midfielder  (MID),  or 
    Forward (FWD). 

**Constraints** - An object is considered a solution if it complies with these conditions:
- Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2 Forwards. 
- Each player is assigned to exactly one team.
- All teams must be complete.
- Each team should not exceed a 750€ million total budget. 
- The average skill rating of the players should be roughly the same among the teams.


# Data Exploration

In [4]:
print(f"This dataset contains {len(df)} players.")
players

This dataset contains 35 players.


[('Alex Carter', 'GK', 85, 90),
 ('Jordan Smith', 'GK', 88, 100),
 ('Ryan Mitchell', 'GK', 83, 85),
 ('Chris Thompson', 'GK', 80, 80),
 ('Blake Henderson', 'GK', 87, 95),
 ('Daniel Foster', 'DEF', 90, 110),
 ('Lucas Bennett', 'DEF', 85, 90),
 ('Owen Parker', 'DEF', 88, 100),
 ('Ethan Howard', 'DEF', 80, 70),
 ('Mason Reed', 'DEF', 82, 75),
 ('Logan Brooks', 'DEF', 86, 95),
 ('Caleb Fisher', 'DEF', 84, 85),
 ('Nathan Wright', 'MID', 92, 120),
 ('Connor Hayes', 'MID', 89, 105),
 ('Dylan Morgan', 'MID', 91, 115),
 ('Hunter Cooper', 'MID', 83, 85),
 ('Austin Torres', 'MID', 82, 80),
 ('Gavin Richardson', 'MID', 87, 95),
 ('Spencer Ward', 'MID', 84, 85),
 ('Sebastian Perry', 'FWD', 95, 150),
 ('Xavier Bryant', 'FWD', 90, 120),
 ('Elijah Sanders', 'FWD', 93, 140),
 ('Adrian Collins', 'FWD', 85, 90),
 ('Tyler Jenkins', 'FWD', 80, 70),
 ('Chase Murphy', 'FWD', 86, 95),
 ('Landon Powell', 'FWD', 89, 110),
 ('Julian Scott', 'FWD', 92, 130),
 ('Bentley Rivera', 'MID', 88, 100),
 ('Maxwell Flores'

In [5]:
num_duplicates = df.duplicated().sum()
print(f"There are {num_duplicates} duplicate rows.")

There are 0 duplicate rows.


In [6]:
print(f"There are {df['Position'].nunique()} unique positions in the dataset.")
df['Position'].value_counts()

There are 4 unique positions in the dataset.


DEF    10
MID    10
FWD    10
GK      5
Name: Position, dtype: int64

In [7]:
df['Rating'].describe()

count    35.000000
mean     86.400000
std       4.095909
min      79.000000
25%      83.500000
50%      86.000000
75%      89.500000
max      95.000000
Name: Rating, dtype: float64

Player ratings vary from 79 to 95. The average rating is 86.4.

In [8]:
df['Performance'].describe()

count     35.000000
mean      97.828571
std       19.876805
min       65.000000
25%       85.000000
50%       95.000000
75%      110.000000
max      150.000000
Name: Performance, dtype: float64

Players costs vary from 65€ million to 150€ million. The average cost for a player is ~98€ million.

# Solution Implementation

## Defining constants

In [9]:
# Constants
NUM_TEAMS = 5
PLAYERS_PER_TEAM = 7
MAX_BUDGET = 750  # Million €
POSITION_REQUIREMENTS = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}

## Defining Solution Function

In [96]:
class TeamAssignmentSolution:
    def __init__(self, players, population=None):
        self.players = players  # List of (name, position, skill, cost)
        self.population = population if population else {}

    def random_initial_representation(self, max_attempts=1000):
        """Randomly create a valid team assignment representation."""
        for attempt in range(max_attempts):
            teams = {i: {"GK": [], "DEF": [], "MID": [], "FWD": [], "total_cost": 0} for i in range(NUM_TEAMS)}
            assigned_players = set()
            shuffled_players = random.sample(self.players, len(self.players))

            for player in shuffled_players:
                name, position, skill, cost = player
                for team_id in sorted(teams, key=lambda tid: teams[tid]["total_cost"]):
                    team = teams[team_id]
                    if len(team[position]) < POSITION_REQUIREMENTS[position] and team["total_cost"] + cost <= MAX_BUDGET:
                        team[position].append(player)
                        team["total_cost"] += cost
                        assigned_players.add(name)
                        break  # Move to next player
                # If no team could take the player, we skip to next (will check validity below)

            # Validate result
            valid_teams = True
            for team in teams.values():
                total_players = sum(len(team[pos]) for pos in POSITION_REQUIREMENTS)
                if total_players != PLAYERS_PER_TEAM:
                    valid_teams = False
                    break
                for pos, required_count in POSITION_REQUIREMENTS.items():
                    if len(team[pos]) != required_count:
                        valid_teams = False
                        break

            if valid_teams and len(assigned_players) == len(self.players):
                self.population = teams
                return self.population

        raise ValueError("Failed to generate a valid team assignment after many attempts.")

    def fitness(self):
        """Evaluate team balance: lower standard deviation of average skill means better balance."""
        if not self.population:
            raise ValueError("Population not initialized. Run random_initial_representation() first.")

        team_avg_skills = []

        for team in self.population.values():
            total_skill = 0
            total_players = 0
            for position in POSITION_REQUIREMENTS:
                for player in team[position]:
                    total_skill += player[2]  # Skill
                    total_players += 1
            avg_skill = total_skill / total_players if total_players > 0 else 0
            team_avg_skills.append(avg_skill)

        return np.std(team_avg_skills)  # Lower is better (balanced teams)


    def print_teams(self):
        for i, team in self.population.items():
            total_skill = 0
            total_players = 0

            for position in ["GK", "DEF", "MID", "FWD"]:
                total_skill += sum(player[2] for player in team[position])
                total_players += len(team[position])

            avg_skill = total_skill / total_players if total_players > 0 else 0

            print(f"\nTeam {i + 1} (Total Cost: {team['total_cost']}M, Average Skill: {avg_skill:.2f})")
            for position in ["GK", "DEF", "MID", "FWD"]:
                for player in team[position]:
                    name, pos, skill, cost = player
                    print(f"  {name} - {pos} | Skill: {skill}, Cost: {cost}M")


In [68]:
solution = TeamAssignmentSolution(players)
solution.random_initial_representation()

{0: {'GK': [('Chris Thompson', 'GK', 80, 80)],
  'DEF': [('Caleb Fisher', 'DEF', 84, 85), ('Owen Parker', 'DEF', 88, 100)],
  'MID': [('Austin Torres', 'MID', 82, 80), ('Dylan Morgan', 'MID', 91, 115)],
  'FWD': [('Landon Powell', 'FWD', 89, 110), ('Colton Gray', 'FWD', 91, 125)],
  'total_cost': 695},
 1: {'GK': [('Ryan Mitchell', 'GK', 83, 85)],
  'DEF': [('Lucas Bennett', 'DEF', 85, 90),
   ('Brayden Hughes', 'DEF', 87, 100)],
  'MID': [('Gavin Richardson', 'MID', 87, 95),
   ('Bentley Rivera', 'MID', 88, 100)],
  'FWD': [('Xavier Bryant', 'FWD', 90, 120),
   ('Elijah Sanders', 'FWD', 93, 140)],
  'total_cost': 730},
 2: {'GK': [('Alex Carter', 'GK', 85, 90)],
  'DEF': [('Daniel Foster', 'DEF', 90, 110),
   ('Maxwell Flores', 'DEF', 81, 72)],
  'MID': [('Connor Hayes', 'MID', 89, 105), ('Spencer Ward', 'MID', 84, 85)],
  'FWD': [('Julian Scott', 'FWD', 92, 130), ('Tyler Jenkins', 'FWD', 80, 70)],
  'total_cost': 662},
 3: {'GK': [('Jordan Smith', 'GK', 88, 100)],
  'DEF': [('Logan B

## Assessing Fitness

In optimization problems, "fitness" is a measure of how good the solution is based on certain criteria. For our approach, we will assess fitness based on the average skill rating of the players on the 5 teams. This will be stored as their skill rating's standard deviation and from now on we intend on gradually improving our teams by swapping players between them to decrease the fitness score (as in a minimization problem).

In [69]:
# Evaluate the fitness of the initial population
fitness_score = solution.fitness()
print(f"Fitness Score: {fitness_score}")
solution.print_teams()

Fitness Score: 0.8494896427039148

Team 1 (Total Cost: 695M, Average Skill: 86.43)
  Chris Thompson - GK | Skill: 80, Cost: 80M
  Caleb Fisher - DEF | Skill: 84, Cost: 85M
  Owen Parker - DEF | Skill: 88, Cost: 100M
  Austin Torres - MID | Skill: 82, Cost: 80M
  Dylan Morgan - MID | Skill: 91, Cost: 115M
  Landon Powell - FWD | Skill: 89, Cost: 110M
  Colton Gray - FWD | Skill: 91, Cost: 125M

Team 2 (Total Cost: 730M, Average Skill: 87.57)
  Ryan Mitchell - GK | Skill: 83, Cost: 85M
  Lucas Bennett - DEF | Skill: 85, Cost: 90M
  Brayden Hughes - DEF | Skill: 87, Cost: 100M
  Gavin Richardson - MID | Skill: 87, Cost: 95M
  Bentley Rivera - MID | Skill: 88, Cost: 100M
  Xavier Bryant - FWD | Skill: 90, Cost: 120M
  Elijah Sanders - FWD | Skill: 93, Cost: 140M

Team 3 (Total Cost: 662M, Average Skill: 85.86)
  Alex Carter - GK | Skill: 85, Cost: 90M
  Daniel Foster - DEF | Skill: 90, Cost: 110M
  Maxwell Flores - DEF | Skill: 81, Cost: 72M
  Connor Hayes - MID | Skill: 89, Cost: 105M
  S

# Selection Mechanism

``Selection Mechanisms`` plays a crucial role in determining which solutions will be passed on to the next generation for our evolutionary algorithm. In this context, it is responsible for choosing which team assignments should undergo further modification to evolve to better solutions. The best solutiond will be more likely to be retained and refined, while less optimal solutions are less likely to survive.

##### `Ranking Selection`

The solutions with better fitness (lower fitness scores in a minimization problem) should be selected more frequently, as their ranks will be higher.

In [70]:
def ranking_selection(population: list, maximization: bool = False):
    """Ranking selection with fitness-based sorting and rank-weighted probabilities."""

    # Sort individuals based on fitness
    sorted_population = sorted(population, key=lambda ind: ind.fitness(), reverse=maximization)

    # Assign ranks: rank 1 = best, rank N = worst
    ranks = list(range(len(sorted_population), 0, -1))  # Higher rank = higher selection chance

    # Convert ranks to probabilities
    total_rank = sum(ranks)
    probabilities = [rank / total_rank for rank in ranks]

    # Roulette-wheel selection
    random_nr = random.uniform(0, 1)
    cumulative_prob = 0

    for idx, prob in enumerate(probabilities):
        cumulative_prob += prob
        if random_nr <= cumulative_prob:
            return deepcopy(sorted_population[idx])

##### `Tournament Selection`

Since the individuals are randomly chosen, the frequency distribution will depend on the tournament size and randomness of the selections.

In [71]:
def tournament_selection(population: list, tournament_size: int = 3, maximization: bool = False):
    """Tournament Selection (Always select the best individual)"""

    # Randomly select a subset of individuals (tournament size)
    tournament_individuals = random.sample(population, tournament_size)

    # Sort the tournament individuals by fitness
    # For maximization: best individual has highest fitness
    # For minimization: best individual has lowest fitness
    if maximization:
        best_individual = max(tournament_individuals, key=lambda ind: ind.fitness())  # Select best for maximization
    else:
        best_individual = min(tournament_individuals, key=lambda ind: ind.fitness())  # Select best for minimization

    return deepcopy(best_individual)  # Return the best individual from the tournament

##### `Selection Results`

In [72]:
def test_selection():
    # Create a small population of TeamAssignmentSolution instances
    population = [TeamAssignmentSolution(players) for _ in range(10)]

    # Ensure that each individual has been initialized properly
    for i, sol in enumerate(population):
        sol.random_initial_representation()
        print(f"Solution {i+1} - Fitness: {sol.fitness()}")

    # Run the selection process multiple times (e.g., 100 times) for both methods
    ranking_selected_solutions = [ranking_selection(population) for _ in range(100)]
    tournament_selected_solutions = [tournament_selection(population) for _ in range(100)]

    # Count how many times each fitness was selected
    ranking_fitness_counter = Counter([sol.fitness() for sol in ranking_selected_solutions])
    tournament_fitness_counter = Counter([sol.fitness() for sol in tournament_selected_solutions])

    # Print the selection frequency for both methods
    print("\nRanking Selection Frequency (100 Selections):")
    for fitness, count in ranking_fitness_counter.items():
        print(f"Fitness {fitness}: Selected {count} times")

    print("\nTournament Selection Frequency (100 Selections):")
    for fitness, count in tournament_fitness_counter.items():
        print(f"Fitness {fitness}: Selected {count} times")

# Run the test
test_selection()

Solution 1 - Fitness: 0.9098104758908946
Solution 2 - Fitness: 0.9363411204670825
Solution 3 - Fitness: 0.8778452283278396
Solution 4 - Fitness: 0.9231711097944874
Solution 5 - Fitness: 1.133533395707275
Solution 6 - Fitness: 0.8050858744917332
Solution 7 - Fitness: 0.7137140569598166
Solution 8 - Fitness: 0.8778452283278417
Solution 9 - Fitness: 1.3862296537952243
Solution 10 - Fitness: 0.7417574277569812

Ranking Selection Frequency (100 Selections):
Fitness 0.7417574277569812: Selected 18 times
Fitness 0.7137140569598166: Selected 19 times
Fitness 0.8778452283278396: Selected 18 times
Fitness 0.9231711097944874: Selected 4 times
Fitness 0.9098104758908946: Selected 6 times
Fitness 0.8050858744917332: Selected 15 times
Fitness 0.8778452283278417: Selected 14 times
Fitness 0.9363411204670825: Selected 3 times
Fitness 1.3862296537952243: Selected 1 times
Fitness 1.133533395707275: Selected 2 times

Tournament Selection Frequency (100 Selections):
Fitness 0.8050858744917332: Selected 13

# Crossover Operators

``Crossover operators`` are very important as they are used to combine parts of two or more "parent" solutions to produce "offspring" solutions, with the goal of exploring the search space more efficiently and generating better solutions.
They mimic the biological process of genetic recombination, where in nature, there is mixing of genetic material of organisms, allowing children to inherit possibly useful traits from both their parents allowing for the exploration of new areas in the solution space.

In [97]:
def calculate_average_skill_rating(team):
    """Calculate the average skill rating for a single team."""
    total_skill = 0
    total_players = 0

    for position in POSITION_REQUIREMENTS:
        for player in team[position]:
            total_skill += player[2]  # Add player skill (index 2 in the tuple)
            total_players += 1  # Count player

    avg_skill_rating = total_skill / total_players if total_players > 0 else 0
    return avg_skill_rating  # Return the average skill of all players in the team

##### `Best Performance Crossover`

Creates a new team by selecting the most efficient players (Performance/Rating)
    from two parent teams, while ensuring constraints are met.

In [114]:
def best_performance_crossover(team1, team2, all_players):
    """Best Performance Crossover between two parent teams to create an offspring"""

    # Combine players from both parent teams and remove duplicates by player name
    combined_players = {p[0]: p for pos in POSITION_REQUIREMENTS for p in team1[pos] + team2[pos]}
    candidate_players = list(combined_players.values())

    # Sort players by efficiency = Performance / Rating (cost/skill)
    candidate_players.sort(key=lambda p: p[3] / p[2], reverse=True)

    new_team = defaultdict(list)
    new_team["total_cost"] = 0

    # Try to fill the team with best performers
    for player in candidate_players:
        pos = player[1]
        cost = player[3]
        if len(new_team[pos]) < POSITION_REQUIREMENTS[pos] and new_team["total_cost"] + cost <= MAX_BUDGET:
            new_team[pos].append(player)
            new_team["total_cost"] += cost

    # Fill missing slots using all_players if some positions are still not filled
    used_names = {p[0] for pos in POSITION_REQUIREMENTS for p in new_team[pos]}
    for pos in POSITION_REQUIREMENTS:
        if len(new_team[pos]) < POSITION_REQUIREMENTS[pos]:
            available = [p for p in all_players if p[1] == pos and p[0] not in used_names]
            # Sort available players by efficiency
            available.sort(key=lambda p: p[3] / p[2], reverse=True)
            for p in available:
                if new_team["total_cost"] + p[3] <= MAX_BUDGET:
                    new_team[pos].append(p)
                    new_team["total_cost"] += p[3]
                    used_names.add(p[0])
                if len(new_team[pos]) == POSITION_REQUIREMENTS[pos]:
                    break

    # Final validation: Check if all positions have the required number of players
    valid = all(len(new_team[pos]) == POSITION_REQUIREMENTS[pos] for pos in POSITION_REQUIREMENTS)
    return new_team if valid else None

In [116]:
def test_bp_crossover():
    # Create parent teams
    parent1 = TeamAssignmentSolution(players)
    parent1.random_initial_representation()  # Initialize with random representation

    parent2 = TeamAssignmentSolution(players)
    parent2.random_initial_representation()  # Initialize with random representation

    # Select random teams from each parent
    team_a = parent1.population[random.choice(list(parent1.population.keys()))]
    team_b = parent2.population[random.choice(list(parent2.population.keys()))]

    # Print original teams and their average skill ratings
    print("Parent 1 Team:")
    for pos in POSITION_REQUIREMENTS:
        print(f"{pos}: {[p[0] for p in team_a[pos]]}")
    print("Total cost:", team_a["total_cost"])
    print("Parent 1 Average Skill Rating:", calculate_average_skill_rating(team_a))  # Average skill rating of Parent 1

    print("\nParent 2 Team:")
    for pos in POSITION_REQUIREMENTS:
        print(f"{pos}: {[p[0] for p in team_b[pos]]}")
    print("Total cost:", team_b["total_cost"])
    print("Parent 2 Average Skill Rating:", calculate_average_skill_rating(team_b))  # Average skill rating of Parent 2

    # Perform best performance crossover
    new_team = best_performance_crossover(team_a, team_b, players)
    
    if new_team:
        print("\nNew Team Successfully Generated with Best-Performance Crossover:")
        # Print the new team and its average skill rating
        for pos in POSITION_REQUIREMENTS:
            print(f"{pos}: {[p[0] for p in new_team[pos]]}")
        print("Total cost:", new_team["total_cost"])

        # Calculate average skill rating of the new team
        print("New Team Average Skill Rating:", calculate_average_skill_rating(new_team))  # Average skill rating of new team
    else:
        print("Failed to generate a valid team.")

# Run the test
test_bp_crossover()

Parent 1 Team:
GK: ['Blake Henderson']
DEF: ['Mason Reed', 'Brayden Hughes']
MID: ['Bentley Rivera', 'Hunter Cooper']
FWD: ['Julian Scott', 'Landon Powell']
Total cost: 695
Parent 1 Average Skill Rating: 86.85714285714286

Parent 2 Team:
GK: ['Blake Henderson']
DEF: ['Logan Brooks', 'Maxwell Flores']
MID: ['Spencer Ward', 'Connor Hayes']
FWD: ['Sebastian Perry', 'Chase Murphy']
Total cost: 697
Parent 2 Average Skill Rating: 86.85714285714286
Failed to generate a valid team.


##### `Random Team Mix Crossover`

Creates a new team by randomly mixing players from two parent teams, ensuring all constraints (positions, budget, and total players) are met.

In [120]:
def team_mix_crossover(team1, team2):
    """Random team mix crossover between two parent teams."""
    
    # Collect all players from both teams, avoiding duplicates by player name
    combined_players = {p[0]: p for pos in POSITION_REQUIREMENTS for p in team1[pos] + team2[pos]}
    player_pool = list(combined_players.values())
    
    # Shuffle the combined player pool for random selection
    random.shuffle(player_pool)

    # Initialize the new team
    new_team = defaultdict(list)
    new_team["total_cost"] = 0

    for player in player_pool:
        name, pos, rating, cost = player
        
        # Ensure that we don't exceed the position requirements or the budget
        if len(new_team[pos]) < POSITION_REQUIREMENTS[pos] and new_team["total_cost"] + cost <= MAX_BUDGET:
            new_team[pos].append(player)
            new_team["total_cost"] += cost

        # Stop early if the team is complete (when all positions are filled)
        total_players = sum(len(new_team[p]) for p in POSITION_REQUIREMENTS)
        if total_players == PLAYERS_PER_TEAM:
            break

    # Final validation: check if the team is complete and valid
    valid = all(len(new_team[pos]) == POSITION_REQUIREMENTS[pos] for pos in POSITION_REQUIREMENTS)
    if valid:
        return new_team
    else:
        return print("Invalid Team Composition.")

In [126]:
def test_team_mix_crossover():
    """
    Test the team mix crossover by randomly selecting teams from two parent solutions
    and performing the crossover to generate a new team.
    """
    # Create parent teams
    parent1 = TeamAssignmentSolution(players)
    parent1.random_initial_representation()  # Initialize with random representation

    parent2 = TeamAssignmentSolution(players)
    parent2.random_initial_representation()  # Initialize with random representation

    # Select random teams from each parent's population
    team_a = parent1.population[random.choice(list(parent1.population.keys()))]
    team_b = parent2.population[random.choice(list(parent2.population.keys()))]

    # Print original teams and their average skill ratings
    print("Parent 1 Team:")
    for pos in POSITION_REQUIREMENTS:
        print(f"{pos}: {[p[0] for p in team_a[pos]]}")
    print("Total cost:", team_a["total_cost"])
    print("Parent 1 Average Skill Rating:", calculate_average_skill_rating(team_a))  # Average skill rating of Parent 1

    print("\nParent 2 Team:")
    for pos in POSITION_REQUIREMENTS:
        print(f"{pos}: {[p[0] for p in team_b[pos]]}")
    print("Total cost:", team_b["total_cost"])
    print("Parent 2 Average Skill Rating:", calculate_average_skill_rating(team_b))  # Average skill rating of Parent 2

    # Perform team mix crossover
    new_team = team_mix_crossover(team_a, team_b)

    # Display the result of the crossover
    if new_team:
        print("\nNew Team Created via Team Mix Crossover:")
        for pos in POSITION_REQUIREMENTS:
            print(f"{pos}: {[p[0] for p in new_team[pos]]}")
        print("Total cost:", new_team["total_cost"])
        print("New Team Average Skill Rating:", calculate_average_skill_rating(new_team))  # Average skill rating of new team
    else:
        print("Crossover failed to produce a valid team.")

# Run the test
test_team_mix_crossover()


Parent 1 Team:
GK: ['Jordan Smith']
DEF: ['Owen Parker', 'Mason Reed']
MID: ['Ashton Phillips', 'Austin Torres']
FWD: ['Zachary Nelson', 'Sebastian Perry']
Total cost: 707
Parent 1 Average Skill Rating: 87.28571428571429

Parent 2 Team:
GK: ['Blake Henderson']
DEF: ['Mason Reed', 'Maxwell Flores']
MID: ['Dylan Morgan', 'Austin Torres']
FWD: ['Landon Powell', 'Zachary Nelson']
Total cost: 639
Parent 2 Average Skill Rating: 85.42857142857143

New Team Created via Team Mix Crossover:
GK: ['Jordan Smith']
DEF: ['Maxwell Flores', 'Mason Reed']
MID: ['Dylan Morgan', 'Ashton Phillips']
FWD: ['Sebastian Perry', 'Landon Powell']
Total cost: 732
New Team Average Skill Rating: 88.0


# Mutation Operators

``Mutation operators`` are a key component in evolutionary algorithms because they allow us to introduce random changes into a solution to maintain genetic diversity in the population and help the algorithm explore new solutions. Mutation mimics natural genetic mutation, which introduces variability and allows for the possibility of exploring new regions of the solution space that might not be reached otherwise.

For our approach, we will consider:
- Position swap mutation between teams (permutation-based)
- Skill Balance Mutation (perturbation + swap)
- Multi Position, Multi Player Swap (permutation-based)
- Budgetting Mutation (perturbation with replacement mutation + swap)

`Mutation Rate`

The mutation rate refers to the probability that a mutation will occur. If the mutation rate is too high, the algorithm may become too random and lose its ability to exploit existing good solutions. If it’s too low, the algorithm may lack the necessary diversity to escape local optima. For our approach we decided to go with a X% rate.

##### `Swap Same Position Between Teams`

This function performs a verbose mutation by swapping one player of the same position between two randomly chosen teams. This simple operator will introduce diversity while guaranteeing role preservation which is a requirement for the solution. We beliebe this will effectively help improve the overall solution without radically altering the population. We are printing both teams’ lineups before and after the swap for clarity.

In [137]:
def mutate_swap_same_position(solution: TeamAssignmentSolution):
    team_ids = list(solution.population.keys())
    t1, t2 = random.sample(team_ids, 2)
    pos = random.choice(["GK", "DEF", "MID", "FWD"])

    if not solution.population[t1][pos] or not solution.population[t2][pos]:
        print(f"No mutation occurred — one of the teams had no players in position: {pos}")
        return

    # Select random players from each team in the specified position
    p1_idx = random.randint(0, len(solution.population[t1][pos]) - 1)
    p2_idx = random.randint(0, len(solution.population[t2][pos]) - 1)

    # Player details (name, position, skill, cost)
    p1 = solution.population[t1][pos][p1_idx]
    p2 = solution.population[t2][pos][p2_idx]

    # Current team costs before swap
    current_cost_t1 = solution.population[t1]["total_cost"]
    current_cost_t2 = solution.population[t2]["total_cost"]

    # Calculate average skill ratings before swap
    avg_skill_t1_before = calculate_average_skill_rating(solution.population[t1])
    avg_skill_t2_before = calculate_average_skill_rating(solution.population[t2])

    # Print the players being swapped for debug purposes
    print(f"Swapping Player: {p1[0]} (Cost: {p1[3]}) <-> {p2[0]} (Cost: {p2[3]})")

    # Print the team details before mutation (with costs and average skill)
    print(f"\nBefore Mutation:")
    print(f"Team {t1} {pos}: {[p[0] for p in solution.population[t1][pos]]} (Total Cost: {current_cost_t1}, Avg Skill: {avg_skill_t1_before:.2f})")
    print(f"Team {t2} {pos}: {[p[0] for p in solution.population[t2][pos]]} (Total Cost: {current_cost_t2}, Avg Skill: {avg_skill_t2_before:.2f})")

    # Swap the players: swap their entire record (name, position, skill, cost)
    solution.population[t1][pos][p1_idx], solution.population[t2][pos][p2_idx] = \
        solution.population[t2][pos][p2_idx], solution.population[t1][pos][p1_idx]

    # Updated team costs after swap
    new_cost_t1 = current_cost_t1 - p1[3] + p2[3]  # Subtract the cost of p1 and add the cost of p2
    new_cost_t2 = current_cost_t2 - p2[3] + p1[3]  # Subtract the cost of p2 and add the cost of p1

    # Check if the swap respects the budget constraint
    if new_cost_t1 > MAX_BUDGET or new_cost_t2 > MAX_BUDGET:
        # Revert the swap if it exceeds the budget constraint
        solution.population[t1][pos][p1_idx], solution.population[t2][pos][p2_idx] = \
            solution.population[t2][pos][p2_idx], solution.population[t1][pos][p1_idx]
        print(f"Mutation failed: Swap would exceed the budget for one of the teams.")
        return

    # Calculate average skill ratings after swap
    avg_skill_t1_after = calculate_average_skill_rating(solution.population[t1])
    avg_skill_t2_after = calculate_average_skill_rating(solution.population[t2])

    # After mutation (print after mutation state):
    print(f"\nAfter Mutation:")
    print(f"Team {t1} {pos}: {[p[0] for p in solution.population[t1][pos]]} (Total Cost: {new_cost_t1}, Avg Skill: {avg_skill_t1_after:.2f})")
    print(f"Team {t2} {pos}: {[p[0] for p in solution.population[t2][pos]]} (Total Cost: {new_cost_t2}, Avg Skill: {avg_skill_t2_after:.2f})")

    # Success message (if no errors in swap)
    print(f"Swap successful between Team {t1} and Team {t2} at position {pos.upper()}.")

# Run the test:
mutate_swap_same_position(solution)


Swapping Player: Sebastian Perry (Cost: 150) <-> Zachary Nelson (Cost: 92)

Before Mutation:
Team 3 FWD: ['Sebastian Perry', 'Adrian Collins'] (Total Cost: 705, Avg Skill: 87.00)
Team 4 FWD: ['Zachary Nelson', 'Chase Murphy'] (Total Cost: 622, Avg Skill: 85.00)

After Mutation:
Team 3 FWD: ['Zachary Nelson', 'Adrian Collins'] (Total Cost: 647, Avg Skill: 85.71)
Team 4 FWD: ['Sebastian Perry', 'Chase Murphy'] (Total Cost: 680, Avg Skill: 86.29)
Swap successful between Team 3 and Team 4 at position FWD.


## ❗ checked até aqui

##### `Skill Balance Mutation`

Aims to balance the total skill ratings between teams. The team with the skill rating's lower standard deviation, where players are roughly within the same skill level, will swap a player (randomly) with the team with the highest standard deviation on skill rating, lowering the disparity between teams. This ensures that teams are balanced, penalizing cases where skill disparity is negatively affecting the fitness function.

In [44]:
def skill_balance_mutation(solution: TeamAssignmentSolution):
    """
    This mutation operator aims to balance the skill variance between teams.
    If one team has significantly higher skill variance than others, it swaps players between
    teams to balance the skill distribution, taking into account not only the average skill
    but also the variance of skill ratings across teams.
    """

    # Print the fitness before mutation
    fitness_before = solution.fitness()
    print(f"\nFitness before mutation: {fitness_before}")

    # Step 1: Calculate skill variance (standard deviation) for each team
    team_variances = {}

    for team_id, team in solution.population.items():
        skills = []
        
        # Collect individual player skills from all positions
        for pos in ["GK", "DEF", "MID", "FWD"]:
            for player in team[pos]:
                skills.append(player[2])  # player[2] is the skill rating
        
        # Calculate the variance (standard deviation) of skills for the team
        skill_variance = np.var(skills) if len(skills) > 1 else 0
        team_variances[team_id] = skill_variance

    # Step 2: Identify teams with the highest and lowest skill variance
    high_variance_team = max(team_variances, key=team_variances.get)  # Team with highest variance
    low_variance_team = min(team_variances, key=team_variances.get)   # Team with lowest variance

    # Print out the skill variance for each team
    print(f"\nSkill variances before mutation:")
    for team_id in solution.population.keys():
        print(f"Team {team_id}: Skill Variance = {team_variances[team_id]:.2f}")

    # Step 3: Swap players between the high and low variance teams
    pos = random.choice(["GK", "DEF", "MID", "FWD"])

    # Ensure that both teams have players in the selected position
    if len(solution.population[high_variance_team][pos]) > 0 and len(solution.population[low_variance_team][pos]) > 0:
        player_from_high_variance = random.choice(solution.population[high_variance_team][pos])
        player_from_low_variance = random.choice(solution.population[low_variance_team][pos])

        # Swap the players between the teams
        solution.population[high_variance_team][pos].remove(player_from_high_variance)
        solution.population[low_variance_team][pos].remove(player_from_low_variance)

        solution.population[high_variance_team][pos].append(player_from_low_variance)
        solution.population[low_variance_team][pos].append(player_from_high_variance)

        print(f"\nSwapped players between Team {high_variance_team} and Team {low_variance_team} at position {pos.upper()}")
        print(f"Player {player_from_high_variance[0]} (Skill: {player_from_high_variance[2]}) swapped with "
              f"Player {player_from_low_variance[0]} (Skill: {player_from_low_variance[2]})")
    else:
        print("No mutation occurred — one of the teams had no players in the selected position.")

    # Step 4: Calculate and print the skill variances of the swapped teams after mutation
    team_variances_after = {}

    for team_id, team in solution.population.items():
        skills = []
        
        # Collect individual player skills from all positions
        for pos in ["GK", "DEF", "MID", "FWD"]:
            for player in team[pos]:
                skills.append(player[2])  # player[2] is the skill rating
        
        # Calculate the variance (standard deviation) of skills for the team
        skill_variance = np.var(skills) if len(skills) > 1 else 0
        team_variances_after[team_id] = skill_variance

    # Print out the skill variances of the swapped teams after mutation
    print(f"\nSkill variances after mutation:")
    print(f"Team {high_variance_team}: Skill Variance = {team_variances_after[high_variance_team]:.2f}")
    print(f"Team {low_variance_team}: Skill Variance = {team_variances_after[low_variance_team]:.2f}")

    # Print the fitness after mutation
    fitness_after = solution.fitness()
    print(f"\nFitness after mutation: {fitness_after}")

# Example usage
# Assuming 'solution' is an instance of the TeamAssignmentSolution class
skill_balance_mutation(solution)



Fitness before mutation: 1.0077252622027684

Skill variances before mutation:
Team 0: Skill Variance = 10.69
Team 1: Skill Variance = 9.92
Team 2: Skill Variance = 22.78
Team 3: Skill Variance = 15.92
Team 4: Skill Variance = 17.10

Swapped players between Team 2 and Team 1 at position FWD
Player Sebastian Perry (Skill: 95) swapped with Player Tyler Jenkins (Skill: 80)

Skill variances after mutation:
Team 2: Skill Variance = 12.98
Team 1: Skill Variance = 17.27

Fitness after mutation: 1.2269091744905103


##### `Multi Position, Multi Player Swap`

This function swaps three players from different positions (GK, DEF, MID, FWD) within a team, ensuring that the team structure (1 GK, 2 DEF, 2 MID, 2 FWD) remains valid, and shuffling the players while preserving the overall team balance.

# ❗❗❗❗❗ WRONG

In [134]:
# Function to calculate team cost (total cost of all players in a team)
def calculate_team_cost(team):
    """Calculate the total cost of a team based on the players."""
    cost = 0
    for pos in team.values():
        if isinstance(pos, list):  # Ensure we are dealing with a list of players
            cost += sum([player[3] for player in pos])  # Assuming player[3] is cost
    return cost

# Function to calculate skill standard deviation for a team
def calculate_team_skill_std_dev(team):
    """Calculate the skill standard deviation of a team."""
    skills = []
    for pos in team.values():
        if isinstance(pos, list):  # Ensure we are dealing with a list of players
            skills.extend([player[2] for player in pos])  # Assuming player[2] is skill rating
    return np.std(skills) if len(skills) > 1 else 0

def mutate_swap_three_players(solution: TeamAssignmentSolution):
    """This mutation operator swaps three players from the same team with players from other teams
    in the same positions while respecting budget and skill standard deviation constraints."""
    
    team_ids = list(solution.population.keys())

    # Randomly select one team
    t1 = random.choice(team_ids)

    # Define positions
    positions = ["GK", "DEF", "MID", "FWD"]

    # Randomly select three distinct positions
    pos1, pos2, pos3 = random.sample(positions, 3)

    # Ensure that the selected team has players in these positions
    if not solution.population[t1][pos1] or not solution.population[t1][pos2] or not solution.population[t1][pos3]:
        print("No mutation occurred — the selected team does not have players in the selected positions.")
        return

    # Select random players from each position of the selected team
    player1_idx = random.randint(0, len(solution.population[t1][pos1]) - 1)
    player2_idx = random.randint(0, len(solution.population[t1][pos2]) - 1)
    player3_idx = random.randint(0, len(solution.population[t1][pos3]) - 1)

    player1 = solution.population[t1][pos1][player1_idx]
    player2 = solution.population[t1][pos2][player2_idx]
    player3 = solution.population[t1][pos3][player3_idx]

    # Select players from other teams for the same positions
    other_teams = [team for team in team_ids if team != t1]

    # Randomly select two other teams (t2 and t3)
    t2 = random.choice(other_teams)
    t3 = random.choice(other_teams)

    # Ensure that the other teams have players in these positions
    if not solution.population[t2][pos1] or not solution.population[t2][pos2] or not solution.population[t2][pos3]:
        print(f"Mutation aborted: Team {t2} doesn't have players in the required positions.")
        return
    if not solution.population[t3][pos1] or not solution.population[t3][pos2] or not solution.population[t3][pos3]:
        print(f"Mutation aborted: Team {t3} doesn't have players in the required positions.")
        return

    # Select players from positions on team t2 and t3
    player2_t2_idx = random.randint(0, len(solution.population[t2][pos1]) - 1)
    player2_t3_idx = random.randint(0, len(solution.population[t3][pos2]) - 1)

    player2_t2 = solution.population[t2][pos1][player2_t2_idx]
    player2_t3 = solution.population[t3][pos2][player2_t3_idx]

    # Print before mutation
    print(f"\nBefore Mutation:")
    print(f"Team {t1} Cost: {calculate_team_cost(solution.population[t1])}")
    print(f"Team {t2} Cost: {calculate_team_cost(solution.population[t2])}")
    print(f"Team {t3} Cost: {calculate_team_cost(solution.population[t3])}")
    print(f"Swapped Players: ")

    # Print the players before mutation
    print(f"Team {t1} {pos1}: {player1[0]} → Team {t2} {pos1}: {player2_t2[0]}")
    print(f"Team {t1} {pos2}: {player2[0]} → Team {t3} {pos2}: {player2_t3[0]}")
    print(f"Team {t1} {pos3}: {player3[0]} → Team {t3} {pos3}: {player2_t3[0]}")

    # Perform the swap (swapping the players directly between the teams and positions)
    solution.population[t1][pos1][player1_idx] = player2_t2
    solution.population[t1][pos2][player2_idx] = player2_t3
    solution.population[t1][pos3][player3_idx] = player2_t3

    # Calculate the costs after mutation
    new_team1_cost = calculate_team_cost(solution.population[t1])
    new_team2_cost = calculate_team_cost(solution.population[t2])
    new_team3_cost = calculate_team_cost(solution.population[t3])

    # Revert the mutation if the budget exceeds
    if new_team1_cost > MAX_BUDGET or new_team2_cost > MAX_BUDGET or new_team3_cost > MAX_BUDGET:
        print(f"Warning: Budget exceeded. Mutation is reverted.")
        
        # Revert back the mutation
        solution.population[t1][pos1][player1_idx] = player1
        solution.population[t1][pos2][player2_idx] = player2
        solution.population[t1][pos3][player3_idx] = player3

        return  # Stop further processing if the mutation is invalid

    # After the mutation, we need to update the skill standard deviations for all teams
    team_std_devs_before = {
        t1: calculate_team_skill_std_dev(solution.population[t1]),
        t2: calculate_team_skill_std_dev(solution.population[t2]),
        t3: calculate_team_skill_std_dev(solution.population[t3])
    }

    # Print skill standard deviation before mutation
    print(f"\nSkill Standard Deviation Before Mutation:")
    for team_id, std_dev in team_std_devs_before.items():
        print(f"Team {team_id}: {std_dev:.2f}")

    # Now, calculate the skill standard deviation after the mutation for all involved teams
    team_std_devs_after = {
        t1: calculate_team_skill_std_dev(solution.population[t1]),
        t2: calculate_team_skill_std_dev(solution.population[t2]),
        t3: calculate_team_skill_std_dev(solution.population[t3])
    }

    # Print skill standard deviation after mutation
    print(f"\nSkill Standard Deviation After Mutation:")
    for team_id, std_dev in team_std_devs_after.items():
        print(f"Team {team_id}: {std_dev:.2f}")

    # Check if the standard deviation change is too drastic
    std_dev_threshold = 2.0  # Example threshold for allowable change in standard deviation

    mutation_reverted = False
    for team_id in [t1, t2, t3]:
        if abs(team_std_devs_after[team_id] - team_std_devs_before[team_id]) > std_dev_threshold:
            print(f"Warning: Standard deviation change exceeded threshold for Team {team_id}. Mutation reverted.")
            mutation_reverted = True
            break  # Stop further processing if the mutation is invalid

    if mutation_reverted:
        # Revert the mutation for all teams involved
        solution.population[t1][pos1][player1_idx] = player1
        solution.population[t1][pos2][player2_idx] = player2
        solution.population[t1][pos3][player3_idx] = player3
        return

    # Print final mutation status
    print("\nMutation successful!")
    print(f"New cost for Team {t1}: {new_team1_cost}, Team {t2}: {new_team2_cost}, Team {t3}: {new_team3_cost}")
    for team_id in [t1, t2, t3]:
        print(f"Skill standard deviations after mutation - Team {team_id}: {team_std_devs_after[team_id]:.2f}")

# Run the mutation function (ensure `solution` is initialized)
mutate_swap_three_players(solution)


Before Mutation:
Team 4 Cost: 654
Team 1 Cost: 610
Team 0 Cost: 600
Swapped Players: 
Team 4 GK: Jaxon Griffin → Team 1 GK: Bentley Rivera
Team 4 MID: Jaxon Griffin → Team 0 MID: Austin Torres
Team 4 FWD: Xavier Bryant → Team 0 FWD: Austin Torres

Skill Standard Deviation Before Mutation:
Team 4: 4.72
Team 1: 3.52
Team 0: 3.19

Skill Standard Deviation After Mutation:
Team 4: 4.72
Team 1: 3.52
Team 0: 3.19

Mutation successful!
New cost for Team 4: 664, Team 1: 610, Team 0: 600
Skill standard deviations after mutation - Team 4: 4.72
Skill standard deviations after mutation - Team 1: 3.52
Skill standard deviations after mutation - Team 0: 3.19


##### `Budgetting Mutation`

This mutation operator aims to balance the total costs of two teams by swapping players between the same positions, if their teams costs differs more than the specifiec threshold. It will introduce diversity and help prevent situations where one team may be overloaded with expensive players, while another has a lower-cost, less-competitive roster.

In [72]:
COST_BALANCE_THRESHOLD = 50  # Define a threshold for cost difference to trigger mutation

In [99]:
def mutate_swap_based_on_cost(solution: TeamAssignmentSolution):
    """This mutation operator swaps players within the same position between two teams to balance their costs."""
    
    team_ids = list(solution.population.keys())
    # Select two random teams
    t1, t2 = random.sample(team_ids, 2)

    # Calculate the total cost of each team
    team1_cost = solution.population[t1]["total_cost"]
    team2_cost = solution.population[t2]["total_cost"]

    # If the costs are similar, no need to mutate
    if abs(team1_cost - team2_cost) < COST_BALANCE_THRESHOLD:
        print(f"Teams {t1} and {t2} already have similar costs. No mutation.")
        return
    
    # Determine the position to swap
    pos = random.choice(["GK", "DEF", "MID", "FWD"])

    # Check if both teams have players in the selected position
    if len(solution.population[t1][pos]) > 0 and len(solution.population[t2][pos]) > 0:
        # Select a random player from each team in the chosen position
        p1_idx = random.randint(0, len(solution.population[t1][pos]) - 1)
        p2_idx = random.randint(0, len(solution.population[t2][pos]) - 1)

        player_from_t1 = solution.population[t1][pos][p1_idx]
        player_from_t2 = solution.population[t2][pos][p2_idx]

        # Print swap details
        print(f"Swapped players between Team {t1} and Team {t2} at position {pos.upper()}:")
        print(f"  {player_from_t1[0]} (Cost: {player_from_t1[3]}) <--> {player_from_t2[0]} (Cost: {player_from_t2[3]})")

        # Perform the swap
        solution.population[t1][pos][p1_idx] = player_from_t2
        solution.population[t2][pos][p2_idx] = player_from_t1

        # Recalculate team costs after swap
        new_team1_cost = solution.population[t1]["total_cost"]
        new_team2_cost = solution.population[t2]["total_cost"]

        # Print new cost information
        print(f"New cost for Team {t1}: {new_team1_cost}")
        print(f"New cost for Team {t2}: {new_team2_cost}")

        # Ensure that neither team exceeds the budget after mutation
        if new_team1_cost > MAX_BUDGET or new_team2_cost > MAX_BUDGET:
            print(f"\nWarning: Team cost exceeds the budget after mutation. Reverting mutation.")
            # Revert the mutation (swap the players back)
            solution.population[t1][pos][p1_idx] = player_from_t1
            solution.population[t2][pos][p2_idx] = player_from_t2
            return  # Invalid mutation if the budget is exceeded

    else:
        print("No mutation occurred — one of the teams had no players in the selected position.")

# Call the function (ensure you have initialized 'solution' before)
mutate_swap_based_on_cost(solution)

Swapped players between Team 3 and Team 2 at position GK:
  Chris Thompson (Cost: 80) <--> Alex Carter (Cost: 90)
New cost for Team 3: 635
New cost for Team 2: 697


# Genetic Algorithm Loop